* Pythonの関数定義は、関数オブジェクトへの参照を変数にセットする実行文である。
このときの変数名は関数名が利用される。

* 変数とは名前空間の属性のようなもの

* Pythonでは、全てのオブジェクトは名前空間である。
dir()は、カレントの名前空間の変数を返す

In [159]:
testVar = 0              # globalでtestVarを作る
def testFunc(x):
    testVar = x          # 関数内(local)でtestVarを作る
    print('locals=', locals())
    return x ** 20
testFunc.testVar = 999   # 関数属性としてtestVarを作る

print(testFunc(2))
print(testFunc.testVar)
print(testVar)

locals= {'testVar': 2, 'x': 2}
1048576
999
0


1. 関数内で作ったtestVarは、localなので関数終了で消えてしまう
2. つまり、永続性ある変数は、globalか属性だけ
3. ３種類の変数は全く別物

In [160]:
def testFunc2():
    global testVar    # global指定すれば、globalが見えるようになる
    print('locals=', locals())
    return testVar
print(testFunc2())

locals= {}
0


In [161]:
testFunc2.other = testFunc
print(testFunc2.other.testVar)

999


* 属性に別関数を保持できる

In [162]:
testFunc.other = testFunc2
t = testFunc
print(t.other.other.other.other.other.other.other.other.other.other.testVar)
print(t.other.other.other.other.other.other.other.other.other.other(2))
print(t.other.other.other.other.other.other.other.other.other())

999
locals= {'testVar': 2, 'x': 2}
1048576
locals= {}
0


* お互いの属性でお互いを保持すれば、行き来できる

In [163]:
gn = '一番外側'
def testFunc3():
    n3 = 'testFunc3の中'
    #print('1st=', testFunc(2))
    def testFunc(x):
        print('n3={} , gn={}'.format(n3, gn))
        return x
    print('2nd=', testFunc(2))
    print('locals=', locals())
testFunc3()

n3=testFunc3の中 , gn=一番外側
2nd= 2
locals= {'testFunc': <function testFunc3.<locals>.testFunc at 0x0000000005C9D048>, 'n3': 'testFunc3の中'}


* 関数内部で定義された関数から変数参照すると、順番に外側スコープに向かって変数を探す

In [164]:
gn = '外側'
def testLocal():
    print('in testLocal', gn)
testLocal()    

in testLocal 外側


* 外側変数を参照するだけならOK

In [165]:
gn = '外側'
def testLocal():
    print('in testLocal', gn)
    gn = '内側'
testLocal()

UnboundLocalError: local variable 'gn' referenced before assignment

* 参照と書き込みが含まれている場合にはローカル変数と判断し、参照がNGに変わる
* つまり関数内で参照されるだけの変数は暗黙グローバルで、関数のどこかで代入されたなら、
  明示的にグローバル宣言しない限り、ローカルである。

In [166]:
gn = '外側'
def testLocal():
    gn = '内側'
    print('in testLocal', gn)
testLocal()
print('outer', gn)

in testLocal 内側
outer 外側


* 順番を変えた。これならローカルだけでOK、外側変数は変更されない

In [167]:
gn = '外側'
def testLocal():
    global gn
    gn = '内側'
    print('in testLocal', gn)
testLocal()
print('outer', gn)

in testLocal 内側
outer 内側


* 外側変数はglobal宣言すると、参照も変更もできる
* 但し、同じ名前のローカルとグローバルを切り替えては使えない。どっちかだけ

In [168]:
gn = '外側'
def testLocal():
    gn = '中間'
    def test1():
        nonlocal gn
        gn = '内側'
        print('in test1', gn)
    print('before test1', gn)
    test1()
    print('after test1', gn)
testLocal()
print('outer', gn)

before test1 中間
in test1 内側
after test1 内側
outer 外側


* 中間変数はnonlocal宣言で、参照も変更もできる

---
* デフォルト引数の使い方
    * イミュータブルなものをデフォルト値にする

In [169]:
def foo(x, y=1):
    return x + y
print(foo(5))

6


* デフォルト引数の罠
    * ミュータブルなものをデフォルト値にすると・・・えっ！！！

In [170]:
def foo(x, y=[]):
    y.append(x)
    return y
print(foo(5))
print(foo(10))

[5]
[5, 10]


* デフォルト値は、関数が定義されたときに一度だけ生成される。

* これを利用して、以下のように計算キャッシュを実装できる

In [171]:
def realCalculate(x, y):
    """ 時間がかかる計算をする """
    return x ** 100000 * y ** 10000

def calculate(x, y, _cache={}):
    if (x,y) in _cache:
        return _cache[(x,y)]
    result = realCalculate(x, y)
    _cache[(x,y)] = result
    return result

* 時間計測してみる

In [175]:
import time
s = time.time()
calculate(777, 999)
print('elapsed=', time.time()-s)
s = time.time()
calculate(777, 999)
print('elapsed=', time.time()-s)

elapsed= 0.10920143127441406
elapsed= 0.0


* セルでの時間計測１

In [176]:
%%timeit
calculate(777, 999)

406 ns ± 3.14 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


* セルでの時間計測２

In [177]:
%%time
calculate(1, 1)

Wall time: 0 ns


1